Requirements
- 1)a -- demean w1, b -- demean w2; 
- 2)a -- 对于之前生成的csv文件100行 取使得n of inequality为max的beta值; b -- 100行的average (n of inequality 也是average); CI 用同样的方式(即如果beta是 arg max 那CI也是用arg max， 如果beta是mean, CI 也是mean);
- 3）bound：+-100;+-500;+-1000

In [208]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
import time
np.set_printoptions(suppress = True)

# Demean

df_w1
- var1 = Relationship_Dum(demean)
- var2 = mi_to_zcta5(demean)
- var3 = FinTechIndicator(demean) * rating_avg(demean)
- var4 = FinTechIndicator(demean) * minority_yelp(demean)


df_w2
- var1 = Relationship_Dum
- var2 = mi_to_zcta5
- var3 = FinTechIndicator(demean) * rating_avg(demean)
- var4 = FinTechIndicator(demean) * minority_yelp(demean)

In [3]:
def Exchange_pairs(df_matched, df_unmatched):
    # t1 = time.time()
    
    df_matched_column = df_matched.columns
    df_matched.columns = df_matched_column + '1m'
    df_unmatched.columns = df_unmatched.columns.str.replace('lender_id', 'lender_id1m')
    df1 = pd.merge(df_matched, df_unmatched, on = 'lender_id1m', how = 'inner')
    
    l = df1.columns[:-5].append([df1.columns[-5:] + '1um'])
    df1.columns = l
    df_matched.columns = df_matched_column
    df2 = pd.merge(df_matched, df1, left_on = 'loan_id', right_on = 'loan_id1um', how = 'inner') 
    
    ll = (df2.columns[:6]+'2m').append(df2.columns[6:])
    df2.columns = ll
    df_unmatched.columns = df_unmatched.columns.str.replace('lender_id1m', 'lender_id')
    df3 = pd.merge(df_unmatched, df2, left_on = ['lender_id','loan_id'], right_on = ['lender_id2m','loan_id1m'], how = 'inner')
    lll = (df3.columns[:6]+'2um').append(df3.columns[6:])
    df3.columns = lll
    
    df_keep = pd.DataFrame()
    for i in range(1, 5):
        name = "value" + str(i)
        df_keep[name] = df3["var"+str(i)+"1m"] + df3["var"+str(i)+"2m"] - df3["var"+str(i)+"1um"] - df3["var"+str(i)+"2um"]
    # t2 = time.time()
    # print("Running time: ", t2-t1)
    return df_keep

In [ ]:
df_org = pd.read_stata('sample_org.dta')
df_org['var1'] = df_org['Relationship_Dum'] - df_org['Relationship_Dum'].mean(axis = 0)
df_org['var2'] = df_org['mi_to_zcta5'] - df_org['mi_to_zcta5'].mean(axis = 0)
df_org['var3'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['rating_avg'] - df_org['rating_avg'].mean(axis = 0))
df_org['var4'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['minority_yelp'] - df_org['minority_yelp'].mean(axis = 0))

df_w1 = pd.DataFrame()
df_w1 = df_org[['lender_id', 'loan_id', 'match']]
df_w1.loc[:, 'var1'] = df_org['var1'].values
df_w1.loc[:, 'var2'] = df_org['var2'].values
df_w1.loc[:, 'var3'] = df_org['var3'].values
df_w1.loc[:, 'var4'] = df_org['var4'].values
df_w1.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values

df_w2 = pd.DataFrame()
df_w2 = df_org[['lender_id', 'loan_id', 'match']]
df_w2.loc[:, 'var1'] = df_org['Relationship_Dum'].values
df_w2.loc[:, 'var2'] = df_org['mi_to_zcta5'].values
df_w2.loc[:, 'var3'] = df_org['var3'].values
df_w2.loc[:, 'var4'] = df_org['var4'].values
df_w2.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values

df_matched = pd.read_stata('sample_matched_NY_demean1.dta').iloc[:,: 7]
df_unmatched = pd.read_stata('sample_unmatched_NY_demean1.dta').iloc[:,: 7]

df_sample = df_w1
# df_sample = df_w2

df_matched = df_sample[df_sample["match"] == 1]
df_unmatched = df_sample[df_sample["match"] == 0]
df_matched.drop(["match"], axis = 1, inplace = True) 
df_unmatched.drop(["match"], axis = 1, inplace = True) 
df_matched = df_matched[df_matched["USPS_ZIP_PREF_STATE"] == "NY"]
df_matched = df_matched[df_matched["yearapproved"] == 2020].iloc[:,: 6]
df_unmatched = df_unmatched[df_unmatched["USPS_ZIP_PREF_STATE"] == "NY"]
df_unmatched = df_unmatched[df_unmatched["yearapproved"] == 2020].iloc[:,: 6]
    
df_keep = Exchange_pairs(df_matched, df_unmatched)

In [10]:
df_org = pd.read_stata('sample_org.dta')
df_org['var1'] = df_org['Relationship_Dum'] - df_org['Relationship_Dum'].mean(axis = 0)
df_org['var2'] = df_org['mi_to_zcta5'] - df_org['mi_to_zcta5'].mean(axis = 0)
df_org['var3'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['rating_avg'] - df_org['rating_avg'].mean(axis = 0))
df_org['var4'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['minority_yelp'] - df_org['minority_yelp'].mean(axis = 0))

df_w1 = pd.DataFrame()
df_w1 = df_org[['lender_id', 'loan_id', 'match']]
df_w1.loc[:, 'var1'] = df_org['var1'].values
df_w1.loc[:, 'var2'] = df_org['var2'].values
df_w1.loc[:, 'var3'] = df_org['var3'].values
df_w1.loc[:, 'var4'] = df_org['var4'].values
df_w1.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values

df_w2 = pd.DataFrame()
df_w2 = df_org[['lender_id', 'loan_id', 'match']]
df_w2.loc[:, 'var1'] = df_org['Relationship_Dum'].values
df_w2.loc[:, 'var2'] = df_org['mi_to_zcta5'].values
df_w2.loc[:, 'var3'] = df_org['var3'].values
df_w2.loc[:, 'var4'] = df_org['var4'].values
df_w2.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values

df_matched = pd.read_stata('sample_matched_NY_demean1.dta').iloc[:,: 7]
df_unmatched = pd.read_stata('sample_unmatched_NY_demean1.dta').iloc[:,: 7]

df_sample = df_w1
'''
df_matched = df_sample[df_sample["match"] == 1]
df_unmatched = df_sample[df_sample["match"] == 0]

df_matched.drop(["match"], axis = 1, inplace = True) 
df_unmatched.drop(["match"], axis = 1, inplace = True) 
df_matched = df_matched[df_matched["USPS_ZIP_PREF_STATE"] == "NY"]
df_matched = df_matched[df_matched["yearapproved"] == 2021].iloc[:,: 6]

df_unmatched = df_unmatched[df_unmatched["USPS_ZIP_PREF_STATE"] == "NY"]
df_unmatched = df_unmatched[df_unmatched["yearapproved"] == 2021].iloc[:,: 6]



df_exchange_pairs = Exchange_pairs(df_matched, df_unmatched)
'''
def Fox_func(num, ratio):
    df_sample = df_w2
    df_matched = df_sample[df_sample["match"] == 1]
    df_unmatched = df_sample[df_sample["match"] == 0]
    df_matched.drop(["match"], axis = 1, inplace = True) 
    df_unmatched.drop(["match"], axis = 1, inplace = True) 
    df_matched = df_matched[df_matched["USPS_ZIP_PREF_STATE"] == "NY"]
    df_matched = df_matched[df_matched["yearapproved"] == 2020].iloc[:,: 6]
    df_unmatched = df_unmatched[df_unmatched["USPS_ZIP_PREF_STATE"] == "NY"]
    df_unmatched = df_unmatched[df_unmatched["yearapproved"] == 2020].iloc[:,: 6]
    
    df_keep = Exchange_pairs(df_matched, df_unmatched)
    
    # df_keep.drop(["value3"], axis = 1, inplace = True) 
    # df_keep.drop(["value4"], axis = 1, inplace = True) 
    
    # df_keep.drop(["value2"], axis = 1) 
    # df_keep.drop(["value3"], axis = 1) 

    def objectfunc(beta, df = df_keep):
        return -sum(df.dot(beta) >=0 )
    t1 = time.time()
    # bounds = [(1, 1.0000000001), (-100, 100), (-100, 100), (-100, 100)]
    # bounds = [(-1.000000001, -1), (-500, 500), (-500, 500)]
    bounds = [(-1000, 1000), (-1000, -1000), (-1000, 1000), (-1000, 1000)]
    # bounds = [(-500, 500), (-500, 500), (-500, 500)]
    # bounds = [(-100, 100), (-100, -100), (-100, 100), (-100, 100)]
    # bounds = [(-500, 500), (-500, -500), (-500, 500), (-500, 500)]
    result = differential_evolution(objectfunc, bounds)
    # print(result)

    r = np.append(result.x, round(-result.fun))
    
    # t2 = time.time()
    # print("Differential Evolution time: ", t2 - t1)
    # print("The number of inequalities satisfied is")
    # print(round(-result.fun))
    # Simulation(num, ratio, df_matched, df_unmatched)
    return r




/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [13]:
df_sample

,lender_id,loan_id,match,var1,var2,var3,var4,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,yearapproved
0,1339.0,27917.0,0.0,0.0,17.322745,-0.024180,0.045413,AGAWAM,MA,2020.0
1,1339.0,97252.0,0.0,0.0,17.322745,0.091452,0.045413,AGAWAM,MA,2020.0
2,1339.0,78177.0,0.0,0.0,17.322745,0.091452,-0.070219,AGAWAM,MA,2020.0
3,3402.0,27917.0,0.0,0.0,10.350215,-0.024180,0.045413,AGAWAM,MA,2020.0
4,3402.0,78177.0,0.0,0.0,10.350215,0.091452,-0.070219,AGAWAM,MA,2020.0
...,...,...,...,...,...,...,...,...,...,...
4117067,1631.0,34117.0,1.0,0.0,0.000000,-0.024180,0.045413,KETCHIKAN,AK,2020.0
4117068,3923.0,34122.0,0.0,0.0,0.000000,1.069299,-0.347326,CRAIG,AK,2020.0
4117069,1631.0,34122.0,1.0,0.0,65.689996,-0.139812,0.045413,CRAIG,AK,2020.0
4117070,1631.0,34131.0,1.0,0.0,61.713988,-0.139812,-0.070219,WRANGELL,AK,2020.0


In [5]:
df_result = pd.DataFrame()
for i in range(0, 100):
    print(i)
    result = Fox_func(100, 0.1)
    df_result = df_result.append([list(result)], ignore_index = True)
df_result.columns = ["Beta_1", "Beta_2", "Beta_3", "Beta_4", "Number of of inequalities satisfied"]

0


/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [14]:
# Don't use argmax or idxmax since if the maximum is achieved in multiple locations, only the first is returned.
num_mean = df_result.mean()
num_max = df_result["Number of of inequalities satisfied"].max()
df_result[df_result["Number of of inequalities satisfied"] == num_max]
df_test = df_result
df_test = df_test.append(df_result[df_result["Number of of inequalities satisfied"] == num_max])
df_test = df_test.append(df_result.mean(axis = 0).rename("mean"))
df_test.to_csv("NY2020;D2;2;0131.csv")

In [3]:
df_org = pd.read_stata('sample_org.dta')
df_org['var1'] = df_org['Relationship_Dum'] - df_org['Relationship_Dum'].mean(axis = 0)
df_org['var2'] = df_org['mi_to_zcta5'] - df_org['mi_to_zcta5'].mean(axis = 0)
df_org['var3'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['rating_avg'] - df_org['rating_avg'].mean(axis = 0))
df_org['var4'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['minority_yelp'] - df_org['minority_yelp'].mean(axis = 0))

df_w1 = pd.DataFrame()
df_w1 = df_org[['lender_id', 'loan_id', 'match']]
df_w1.loc[:, 'var1'] = df_org['var1'].values
df_w1.loc[:, 'var2'] = df_org['var2'].values
df_w1.loc[:, 'var3'] = df_org['var3'].values
df_w1.loc[:, 'var4'] = df_org['var4'].values
df_w1.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values

df_w2 = pd.DataFrame()
df_w2 = df_org[['lender_id', 'loan_id', 'match']]
df_w2.loc[:, 'var1'] = df_org['Relationship_Dum'].values
df_w2.loc[:, 'var2'] = df_org['mi_to_zcta5'].values
df_w2.loc[:, 'var3'] = df_org['var3'].values
df_w2.loc[:, 'var4'] = df_org['var4'].values
df_w2.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values

df_sample = df_w2
df_matched = df_sample[df_sample["match"] == 1]
df_unmatched = df_sample[df_sample["match"] == 0]
df_matched.drop(["match"], axis = 1, inplace = True) 
df_unmatched.drop(["match"], axis = 1, inplace = True) 
df_matched = df_matched[df_matched["USPS_ZIP_PREF_STATE"] == "NY"]
df_matched = df_matched[df_matched["yearapproved"] == 2020].iloc[:,: 6]
df_unmatched = df_unmatched[df_unmatched["USPS_ZIP_PREF_STATE"] == "NY"]
df_unmatched = df_unmatched[df_unmatched["yearapproved"] == 2020].iloc[:,: 6]
    

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https

In [ ]:
# num is the # of random samples(or the # of simulations)
# ratio is the random sampling ratio, which is 10% here
def Simulation(num, ratio, df_matched, df_unmatched):
    df_result = pd.DataFrame()
    for i in range(0, num):
        # df_matched_sample = Subsample(df_matched, ratio)
        # df_unmatched_sample = Subsample(df_unmatched, ratio)
        
        loan_id = df_unmatched["loan_id"].unique()
        sample_loan_id = np.random.choice(loan_id, round(loan_id.shape[0] * ratio), replace = False)
        df_unmatched_sample = df_unmatched.loc[df_unmatched["loan_id"].isin(sample_loan_id)]
        df_matched_sample = df_matched.loc[df_matched["loan_id"].isin(sample_loan_id)]
       
        df_keep = Exchange_pairs(df_matched_sample, df_unmatched_sample)
        # bounds = [(1, 1.000000001), (-100, 100), (-100, 100), (-100, 100), (-100, 100)] # fix beta_1 = 1
        # bounds = [(1, 1.0000000001), (-100, 100), (-100, 100), (-100, 100)]
        # bounds = [(-1.000000001, -1), (-500, 500), (-500, 500)]
        bounds = [(-1000, 1000), (-1000, -1000), (-1000, 1000), (-1000, 1000)]
        # bounds = [(-500, 500), (-500, 500), (-500, 500)]
        # bounds = [(-100, 100), (-100, -100), (-100, 100), (-100, 100)]
        # bounds = [(-500, 500), (-500, -500), (-500, 500), (-500, 500)]
        def objectfunc(beta, df = df_keep):
            return -sum(df.dot(beta) >=0 )
        result = differential_evolution(objectfunc, bounds)
        
        r = np.append(result.x, round(-result.fun))
        
        df_result = df_result.append([list(r)], ignore_index = True)
        # print(i)
    # print("for loop finished")
    df_result.columns = ["Beta_1", "Beta_2", "Beta_3", "Beta_4", "Number of of inequalities satisfied"]
    
    num_mean = df_result.mean()
    num_max = df_result["Number of of inequalities satisfied"].max()
    df_result[df_result["Number of of inequalities satisfied"] == num_max]
    # df_test = df_result
    df_test = pd.DataFrame()
    df_test = df_test.append(df_result[df_result["Number of of inequalities satisfied"] == num_max])
    df_test = df_test.append(df_result.mean(axis = 0).rename("mean"))
    
    df_test.index = pd.Series(["argmax", "mean"])
    
    '''
    print("The 5% quantile of parameters are")
    print(df_result.quantile(0.05))
    print("The 95% quantile of parameters are")
    print(df_result.quantile(0.95))
    '''
    
    return df_test



In [ ]:
t1 = time.time()
df_simulation = pd.DataFrame()

for j in range(0, 1000):
    if j%10 == 0:
        print(j)
    df_result = Simulation(100, 0.1, df_matched, df_unmatched)
    df_simulation = df_simulation.append(df_result)

t2 = time.time()
print("Simulation time: ", t2-t1)


0


/opt/conda/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx


10
20
30
40
50
60
70
80
90
100
110
120
130
140
330
340
350
370


In [216]:
df_simulation.to_csv("Simulation_temp.csv")

In [277]:
df_simulation.to_csv("NY2020;Simulation;D2;2;0227.csv")

In [276]:
df_simulation.loc["argmax"]

,Beta_1,Beta_2,Beta_3,Beta_4,Number of of inequalities satisfied
argmax,364.491321,-500.0,-426.703895,464.184568,65333.0
argmax,423.154410,-500.0,72.384652,467.904722,63338.0
argmax,428.245742,-500.0,151.333138,347.970700,63592.0
argmax,430.610743,-500.0,-498.186426,484.522868,63262.0
argmax,406.027309,-500.0,127.161653,488.479870,68447.0
...,...,...,...,...,...
argmax,453.618612,-500.0,-377.629249,419.641761,66357.0
argmax,421.343776,-500.0,-199.793932,448.088632,64853.0
argmax,470.253521,-500.0,-183.742406,362.329910,65539.0
argmax,414.409256,-500.0,-211.800499,407.769768,67160.0


1)N of inequ这里，排名，取第25名，然后找到对应的beta。
2)对于每个beta分开取的，直接用quantile

In [278]:
df_output = pd.DataFrame()
df_output = df_output.append(df_simulation.loc["argmax"].quantile(0.01))
df_output = df_output.append(df_simulation.loc["argmax"].quantile(0.05))
df_output = df_output.append(df_simulation.loc["argmax"].quantile(0.10))
df_output = df_output.append(df_simulation.loc["argmax"].quantile(0.90))
df_output = df_output.append(df_simulation.loc["argmax"].quantile(0.95))
df_output = df_output.append(df_simulation.loc["argmax"].quantile(0.99))
df_output = df_output.append(df_simulation.loc["mean"].quantile(0.01))
df_output = df_output.append(df_simulation.loc["mean"].quantile(0.05))
df_output = df_output.append(df_simulation.loc["mean"].quantile(0.10))
df_output = df_output.append(df_simulation.loc["mean"].quantile(0.90))
df_output = df_output.append(df_simulation.loc["mean"].quantile(0.95))
df_output = df_output.append(df_simulation.loc["mean"].quantile(0.99))
df_output

,Beta_1,Beta_2,Beta_3,Beta_4,Number of of inequalities satisfied
0.01,178.360669,-500.0,-497.128301,-432.366637,62743.8400
0.05,294.270322,-500.0,-481.382506,1.499188,63326.8000
0.10,339.603245,-500.0,-464.139620,180.797222,63786.0000
0.90,491.971869,-500.0,381.411080,488.058813,67802.5000
0.95,495.199450,-500.0,442.711485,494.175099,68509.0000
0.99,498.960944,-500.0,489.474245,498.217255,69905.4500
0.01,411.201943,-500.0,-107.248631,292.098566,56146.9185
0.05,417.613835,-500.0,-84.520545,306.989801,56442.2075
0.10,420.614219,-500.0,-72.662028,313.835785,56541.7700
0.90,439.647638,-500.0,6.938079,364.937431,57372.8840


In [279]:
df_argmax = df_simulation.loc["argmax"]
df_mean = df_simulation.loc["mean"]
df1 = df_argmax[df_argmax["Number of of inequalities satisfied"] == df_argmax["Number of of inequalities satisfied"].sort_values()[9]]
df_output = df_output.append(df1)
df2 = df_argmax[df_argmax["Number of of inequalities satisfied"] == df_argmax["Number of of inequalities satisfied"].sort_values()[49]]
df_output = df_output.append(df2)
df3 = df_argmax[df_argmax["Number of of inequalities satisfied"] == df_argmax["Number of of inequalities satisfied"].sort_values()[99]]
df_output = df_output.append(df3)
df4 = df_argmax[df_argmax["Number of of inequalities satisfied"] == df_argmax["Number of of inequalities satisfied"].sort_values()[899]]
df_output = df_output.append(df4)
df5 = df_argmax[df_argmax["Number of of inequalities satisfied"] == df_argmax["Number of of inequalities satisfied"].sort_values()[949]]
df_output = df_output.append(df5)
df6 = df_argmax[df_argmax["Number of of inequalities satisfied"] == df_argmax["Number of of inequalities satisfied"].sort_values()[989]]
df_output = df_output.append(df6)

In [280]:
df1 = df_mean[df_mean["Number of of inequalities satisfied"] == df_mean["Number of of inequalities satisfied"].sort_values()[9]]
df_output = df_output.append(df1)
df2 = df_mean[df_mean["Number of of inequalities satisfied"] == df_mean["Number of of inequalities satisfied"].sort_values()[49]]
df_output = df_output.append(df2)
df3 = df_mean[df_mean["Number of of inequalities satisfied"] == df_mean["Number of of inequalities satisfied"].sort_values()[99]]
df_output = df_output.append(df3)
df4 = df_mean[df_mean["Number of of inequalities satisfied"] == df_mean["Number of of inequalities satisfied"].sort_values()[899]]
df_output = df_output.append(df4)
df5 = df_mean[df_mean["Number of of inequalities satisfied"] == df_mean["Number of of inequalities satisfied"].sort_values()[949]]
df_output = df_output.append(df5)
df6 = df_mean[df_mean["Number of of inequalities satisfied"] == df_mean["Number of of inequalities satisfied"].sort_values()[989]]
df_output = df_output.append(df6)

In [281]:
df_simulation.to_csv("NY2020;Simulation;D2;2;0227.csv")
df_output.to_csv("NY2020;Result;D2;2;0227.csv")

In [239]:
df_output

,Beta_1,Beta_2,Beta_3,Beta_4,Number of of inequalities satisfied
0.01,43.029013,-100.0,-98.811193,-89.154596,62929.5200
0.05,60.253665,-100.0,-95.375405,-25.376446,63380.9500
0.1,70.884937,-100.0,-91.172936,25.475582,63770.4000
0.9,98.445446,-100.0,74.612140,97.014122,67810.4000
0.95,99.106805,-100.0,90.800381,98.265524,68543.1000
0.99,99.762228,-100.0,98.456109,99.518228,70371.2700
0.01,82.372831,-100.0,-21.008522,57.618076,56132.3445
0.05,83.478639,-100.0,-16.817538,61.115666,56454.4830
0.1,84.064256,-100.0,-14.720521,62.500499,56557.8710
0.9,87.800807,-100.0,1.309006,73.357585,57422.2680


In [258]:
df_simulation1 = df_simulation

In [264]:
df_simulation2 = df_simulation

In [267]:
df_simulation3 = df_simulation

In [116]:
print("The 10% quantile of parameters are")
print(df_simulation.loc["argmax"].quantile(0.10))
print("The 90% quantile of parameters are")
print(df_simulation.loc["argmax"].quantile(0.90))

The 10% quantile of parameters are
Beta_1                                   358.420266
Beta_2                                  -500.000000
Beta_3                                  -468.969329
Beta_4                                   153.651074
Number of of inequalities satisfied    63875.300000
Name: 0.1, dtype: float64
The 90% quantile of parameters are
Beta_1                                   492.356307
Beta_2                                  -500.000000
Beta_3                                   388.880781
Beta_4                                   485.522528
Number of of inequalities satisfied    67904.900000
Name: 0.9, dtype: float64


In [ ]:
a = [1,2,3,4,5,6,7,8,9,10]

In [ ]:
np.quantile(a, 0.555)

In [1]:
df_simulation

NameError: name 'df_simulation' is not defined

# CounterFactual

In [37]:
df_beta = pd.read_csv("NY2020;Result;D2;2;0227.csv").iloc[:, 1:5]
df_beta.head()

,Beta_1,Beta_2,Beta_3,Beta_4
0,178.360669,-500.0,-497.128301,-432.366637
1,294.270322,-500.0,-481.382506,1.499188
2,339.603245,-500.0,-464.139620,180.797222
3,491.971869,-500.0,381.411080,488.058813
4,495.199450,-500.0,442.711485,494.175099


In [338]:
df_org = pd.read_stata('sample_org.dta')
df_org['var1'] = df_org['Relationship_Dum'] - df_org['Relationship_Dum'].mean(axis = 0)
df_org['var2'] = df_org['mi_to_zcta5'] - df_org['mi_to_zcta5'].mean(axis = 0)
df_org['var3'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['rating_avg'] - df_org['rating_avg'].mean(axis = 0))
df_org['var4'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['minority_yelp'] - df_org['minority_yelp'].mean(axis = 0))

'''
df_w1 = pd.DataFrame()
df_w1 = df_org[['lender_id', 'loan_id', 'match']]
df_w1.loc[:, 'var1'] = df_org['var1'].values
df_w1.loc[:, 'var2'] = df_org['var2'].values
df_w1.loc[:, 'var3'] = df_org['var3'].values
df_w1.loc[:, 'var4'] = df_org['var4'].values
df_w1.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values
'''

df_w2 = pd.DataFrame()
df_w2 = df_org[['lender_id', 'loan_id', 'match']]
df_w2.loc[:, 'var1'] = df_org['Relationship_Dum'].values
df_w2.loc[:, 'var2'] = df_org['mi_to_zcta5'].values
df_w2.loc[:, 'var3'] = df_org['var3'].values
df_w2.loc[:, 'var4'] = df_org['var4'].values
df_w2.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values

df_sample = df_w2
'''
df_matched = df_sample[df_sample["match"] == 1]
df_unmatched = df_sample[df_sample["match"] == 0]
df_matched.drop(["match"], axis = 1, inplace = True) 
df_unmatched.drop(["match"], axis = 1, inplace = True) 
df_matched = df_matched[df_matched["USPS_ZIP_PREF_STATE"] == "NY"]
df_matched = df_matched[df_matched["yearapproved"] == 2020].iloc[:,: 6]
df_unmatched = df_unmatched[df_unmatched["USPS_ZIP_PREF_STATE"] == "NY"]
df_unmatched = df_unmatched[df_unmatched["yearapproved"] == 2020].iloc[:,: 6]
'''
# df_sample.drop(["match"], axis = 1, inplace = True) 
df_sample = df_sample[df_sample["USPS_ZIP_PREF_STATE"] == "NY"]
df_sample = df_sample[df_sample["yearapproved"] == 2020].iloc[:,: 7]
df_matched = df_sample[df_sample["match"] == 1]
df_unmatched = df_sample[df_sample["match"] == 0]
df_matched.drop(["match"], axis = 1, inplace = True) 
df_unmatched.drop(["match"], axis = 1, inplace = True) 

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https

In [387]:
df_org_sample = df_org[df_org["USPS_ZIP_PREF_STATE"] == "NY"]
df_org_sample = df_org_sample[df_org_sample["yearapproved"] == 2020]
df_org_sample

,lender_id,loan_id,rating_avg,minority_yelp,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,FinTechIndicator,Relationship_Dum,match,mi_to_zcta5,yearapproved,var1,var2,var3,var4
87862,814.0,19445.0,5.0,0.0,FISHERS ISLAND,NY,0,0.0,1.0,11.481559,2020.0,-0.001122,-549.104524,-0.139812,0.045413
156347,561.0,29072.0,4.0,0.0,NEW YORK,NY,0,0.0,0.0,123.103606,2020.0,-0.001122,-437.482477,-0.024180,0.045413
156348,561.0,77093.0,2.4,0.0,NEW YORK,NY,0,0.0,0.0,123.103606,2020.0,-0.001122,-437.482477,0.160831,0.045413
156349,561.0,49444.0,4.0,0.0,NEW YORK,NY,0,0.0,0.0,123.103606,2020.0,-0.001122,-437.482477,-0.024180,0.045413
156350,561.0,77470.0,4.0,0.0,NEW YORK,NY,0,0.0,0.0,123.103606,2020.0,-0.001122,-437.482477,-0.024180,0.045413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599443,3278.0,36138.0,4.5,0.0,ELMIRA,NY,0,0.0,0.0,174.711647,2020.0,-0.001122,-385.874436,-0.081996,0.045413
599444,697.0,36138.0,4.5,0.0,ELMIRA,NY,0,0.0,0.0,1279.236655,2020.0,-0.001122,718.650572,-0.081996,0.045413
599445,697.0,19525.0,1.0,1.0,ELMIRA,NY,0,0.0,0.0,1279.236655,2020.0,-0.001122,718.650572,0.322716,-0.070219
599446,3923.0,36138.0,4.5,0.0,ELMIRA,NY,1,0.0,0.0,0.000000,2020.0,-0.001122,-560.586083,0.627114,-0.347326


In [388]:
df_org_sample["FinTechIndicator"].sum()

37865

In [385]:
df_sample["FinTechIndicator"] = df_org_sample["FinTechIndicator"]
df_sample["MinorityIndicator"] = df_org_sample["minority_yelp"]
df_sample

,lender_id,loan_id,match,var1,var2,var3,var4,FinTechIndicator,MinorityIndicator
87862,814.0,19445.0,1.0,0.0,11.481559,-0.139812,0.045413,0,0.0
156347,561.0,29072.0,0.0,0.0,123.103606,-0.024180,0.045413,0,0.0
156348,561.0,77093.0,0.0,0.0,123.103606,0.160831,0.045413,0,0.0
156349,561.0,49444.0,0.0,0.0,123.103606,-0.024180,0.045413,0,0.0
156350,561.0,77470.0,0.0,0.0,123.103606,-0.024180,0.045413,0,0.0
...,...,...,...,...,...,...,...,...,...
599443,3278.0,36138.0,0.0,0.0,174.711647,-0.081996,0.045413,0,0.0
599444,697.0,36138.0,0.0,0.0,1279.236655,-0.081996,0.045413,0,0.0
599445,697.0,19525.0,0.0,0.0,1279.236655,0.322716,-0.070219,0,1.0
599446,3923.0,36138.0,0.0,0.0,0.000000,0.627114,-0.347326,1,0.0


In [347]:
df_pred = df_sample.copy()
df_pred["FinTechIndicator"] = df_org_sample["FinTechIndicator"]
df_pred["MinorityIndicator"] = df_org_sample["minority_yelp"]
df_pred

,lender_id,loan_id,match,var1,var2,var3,var4,FinTechIndicator,MinorityIndicator
87862,814.0,19445.0,1.0,0.0,11.481559,-0.139812,0.045413,0,0.0
156347,561.0,29072.0,0.0,0.0,123.103606,-0.024180,0.045413,0,0.0
156348,561.0,77093.0,0.0,0.0,123.103606,0.160831,0.045413,0,0.0
156349,561.0,49444.0,0.0,0.0,123.103606,-0.024180,0.045413,0,0.0
156350,561.0,77470.0,0.0,0.0,123.103606,-0.024180,0.045413,0,0.0
...,...,...,...,...,...,...,...,...,...
599443,3278.0,36138.0,0.0,0.0,174.711647,-0.081996,0.045413,0,0.0
599444,697.0,36138.0,0.0,0.0,1279.236655,-0.081996,0.045413,0,0.0
599445,697.0,19525.0,0.0,0.0,1279.236655,0.322716,-0.070219,0,1.0
599446,3923.0,36138.0,0.0,0.0,0.000000,0.627114,-0.347326,1,0.0


In [386]:
df_obs = pd.DataFrame()
df_obs.loc["obs", "% of minority using fintech"] = df_sample[df_sample["MinorityIndicator"] == 1]["FinTechIndicator"].sum()/df_sample.shape[0]
df_obs.loc["obs", "% of non-minority using fintech"] = df_sample[df_sample["MinorityIndicator"] == 0]["FinTechIndicator"].sum()/df_sample.shape[0]
df_obs.loc["obs", "% of minority using fintech - % of non-minority using fintech"] = df_obs.loc["obs", "% of minority using fintech"] - df_obs.loc["obs", "% of non-minority using fintech"]
df_obs

,% of minority using fintech,% of non-minority using fintech,% of minority using fintech - % of non-minority using fintech
obs,0.038966,0.054701,-0.015735


In [352]:
betas = np.array(df_beta.loc[i].values).reshape(4,1)
df_pred["ShutoffBeta4"] = df_pred.iloc[:, 2:5].dot(betas[0:3])
b = np.array([[1],[1],[1],[-1]]) # shape = (4,1)
# df_pred["pred"] = df_pred.iloc[:, 2:6].dot(betas) >= 0
df_pred["ReverseBeta4"] = df_pred.iloc[:, 2:6].dot(np.multiply(betas, b)) # Hadamard Product
df_pred["ShutoffBeta1"] = df_pred.iloc[:, 3:6].dot(betas[1:4])
df_pred["ShutoffBeta2"] = df_pred.iloc[:, [2, 4, 5]].dot(betas[[0, 2, 3]])
df_pred["ShutoffBeta3"] = df_pred.iloc[:, [2, 3, 5]].dot(betas[[0, 1, 3]])

In [380]:
# Calculate lender's quota 
df_quota = pd.DataFrame(df_pred["lender_id"].value_counts())
df_quota.columns = ["quota"]
df_quota["used"] = 0

In [381]:
df_quota

,quota,used
2605.0,778,0
3923.0,586,0
5000.0,576,0
1224.0,575,0
1797.0,470,0
...,...,...
57.0,1,0
2668.0,1,0
3695.0,1,0
814.0,1,0


In [356]:
for item in ["ShutoffBeta4", "ReverseBeta4", "ShutoffBeta1", "ShutoffBeta2", "ShutoffBeta3"]:
    df_pred.sort_values(by = [item], axis = 0, ascending = False, inplace = True)
    for index, row in df_pred.iterrows():
        if df_quota.loc[row["lender_id"], "used"] < df_quota.loc[row["lender_id"], "quota"]:
            df_quota.loc[row["lender_id"], "used"] += 1
            # Assign borrower, then delete the borrower
            df_pred.drop(df_pred[df_pred["loan_id"] == row["loan_id"]].index.delete(0), inplace = True)
        else:
            # Delete lender since quota is full
            df_pred.drop(df_pred[df_pred["lender_id"] == row["lender_id"]].index, inplace = True)

In [382]:
df_pred.sort_values(by = ["ReverseBeta4"], axis = 0, ascending = False, inplace = True)
df_pred

,lender_id,loan_id,match,var1,var2,var3,var4,FinTechIndicator,MinorityIndicator,ShutoffBeta4,ReverseBeta4,ShutoffBeta1,ShutoffBeta2,ShutoffBeta3
576219,2605.0,28292.0,1.0,0.0,0.000000,1.069299,0.537042,1,1.0,178.360669,640.689666,-462.328998,-283.968329,-283.968329
570222,1224.0,25707.0,1.0,0.0,0.000000,1.069299,-0.347326,1,0.0,178.360669,640.689666,-462.328998,-283.968329,-283.968329
586254,2569.0,47018.0,1.0,0.0,0.000000,1.069299,0.537042,1,1.0,178.360669,640.689666,-462.328998,-283.968329,-283.968329
597618,1224.0,26435.0,1.0,0.0,0.000000,1.069299,-0.347326,1,0.0,178.360669,640.689666,-462.328998,-283.968329,-283.968329
549652,736.0,18140.0,1.0,0.0,0.000000,1.069299,0.537042,1,1.0,178.360669,640.689666,-462.328998,-283.968329,-283.968329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572359,2630.0,53053.0,1.0,0.0,433.782947,-0.116685,0.045413,0,0.0,-215467.418667,-215517.869539,-215595.328465,-215416.967796,228.811540
403304,2572.0,50635.0,1.0,0.0,470.713203,-0.116685,0.045413,0,0.0,-233826.494453,-233876.945324,-233954.404251,-233776.043582,228.811540
403305,2572.0,95804.0,0.0,0.0,470.713203,-0.119643,0.045413,0,0.0,-234004.855122,-234056.584943,-233953.125301,-233953.125301,51.729821
551059,2572.0,49555.0,1.0,0.0,495.146589,0.168540,0.045413,0,0.0,-245973.021967,-245900.150861,-246224.253742,-246045.893073,105.489563


In [383]:
t1 = time.time()
for index, row in df_pred.iterrows():
    if df_quota.loc[row["lender_id"], "used"] < df_quota.loc[row["lender_id"], "quota"]:
        df_quota.loc[row["lender_id"], "used"] += 1
        # Assign borrower, then delete the borrower
        df_pred.drop(df_pred[df_pred["loan_id"] == row["loan_id"]].index.delete(0), inplace = True)
    else:
        # Delete lender since quota is full
        df_pred.drop(df_pred[df_pred["lender_id"] == row["lender_id"]].index, inplace = True)
t2 = time.time()
print("Assign time: ", t2 - t1)     

Assign time:  25.385658502578735


In [375]:
# Assign time:  3077.200585126877
df_pred1 = df_pred.copy()

In [370]:
df_pred

,lender_id,loan_id,match,var1,var2,var3,var4,FinTechIndicator,MinorityIndicator,ShutoffBeta4,ReverseBeta4,ShutoffBeta1,ShutoffBeta2,ShutoffBeta3
547743,2605.0,27432.0,1.0,0.0,0.000000,-2.468174,0.537042,1,1.0,178.360669,-888.795321,1067.155989,1245.516658,1245.516658
568728,425.0,13338.0,1.0,0.0,0.000000,0.062544,0.045413,0,0.0,178.360669,205.402663,-27.041994,151.318675,151.318675
458305,1224.0,25191.0,1.0,0.0,0.000000,0.569891,0.537042,1,1.0,178.360669,424.762387,-246.401718,-68.041049,-68.041049
568899,1374.0,31601.0,1.0,0.0,0.000000,0.041896,0.045413,0,0.0,178.360669,196.474913,-18.114244,160.246425,160.246425
388429,2605.0,27310.0,1.0,0.0,0.000000,-1.490714,0.537042,1,1.0,178.360669,-466.174440,644.535109,822.895778,822.895778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572359,2630.0,53053.0,1.0,0.0,433.782947,-0.116685,0.045413,0,0.0,-215467.418667,-215517.869539,-215595.328465,-215416.967796,228.811540
403304,2572.0,50635.0,1.0,0.0,470.713203,-0.116685,0.045413,0,0.0,-233826.494453,-233876.945324,-233954.404251,-233776.043582,228.811540
403305,2572.0,95804.0,0.0,0.0,470.713203,-0.119643,0.045413,0,0.0,-234004.855122,-234056.584943,-233953.125301,-233953.125301,51.729821
551059,2572.0,49555.0,1.0,0.0,495.146589,0.168540,0.045413,0,0.0,-245973.021967,-245900.150861,-246224.253742,-246045.893073,105.489563


In [373]:
# Shut off Beta4
df_temp = df_pred[(df_pred["MinorityIndicator"] == 1) & (df_pred["ShutoffBeta4"] >= 0)]
df_obs.loc["ShutoffBeta4", "% of minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
df_temp = df_pred[(df_pred["MinorityIndicator"] == 0) & (df_pred["ShutoffBeta4"] >= 0)]
df_obs.loc["ShutoffBeta4", "% of non-minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
df_obs.loc["ShutoffBeta4", "% of minority using fintech - % of non-minority using fintech"] = df_obs.loc["ShutoffBeta4", "% of minority using fintech"] - df_obs.loc["ShutoffBeta4", "% of non-minority using fintech"] 
df_obs.loc["ShutoffBeta4", "Obs. – Counter."] = df_obs.loc["ShutoffBeta4", "% of minority using fintech - % of non-minority using fintech"] - df_obs.loc["obs", "% of minority using fintech - % of non-minority using fintech"] 

In [205]:
# for i in range(0, df_beta.shape[0]):
for i in range(0, 1):
    betas = np.array(df_beta.loc[i].values).reshape(4,1)
    b = np.array([[1],[1],[1],[-1]]) # shape = (4,1)
    # df_pred["pred"] = df_pred.iloc[:, 2:6].dot(betas) >= 0
    '''
    df_pred["ShutoffBeta4"] = df_pred.iloc[:, 2:5].dot(betas[0:3]) >= 0
    df_pred["ReverseBeta4"] = df_pred.iloc[:, 2:6].dot(np.multiply(betas, b)) >= 0 # Hadamard Product
    df_pred["ShutoffBeta1"] = df_pred.iloc[:, 3:6].dot(betas[1:4]) >= 0
    df_pred["ShutoffBeta2"] = df_pred.iloc[:, [2, 4, 5]].dot(betas[[0, 2, 3]]) >= 0
    df_pred["ShutoffBeta3"] = df_pred.iloc[:, [2, 3, 5]].dot(betas[[0, 1, 3]]) >= 0
    '''
    
    # Shut off Beta4
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 1) & (df_pred["ShutoffBeta4"] >= 0)]
    df_obs.loc["ShutoffBeta4", "% of minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 0) & (df_pred["ShutoffBeta4"] >= 0)]
    df_obs.loc["ShutoffBeta4", "% of non-minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_obs.loc["ShutoffBeta4", "% of minority using fintech - % of non-minority using fintech"] = df_obs.loc["ShutoffBeta4", "% of minority using fintech"] - df_obs.loc["ShutoffBeta4", "% of non-minority using fintech"] 
    df_obs.loc["ShutoffBeta4", "Obs. – Counter."] = df_obs.loc["ShutoffBeta4", "% of minority using fintech - % of non-minority using fintech"] - df_obs.loc["obs", "% of minority using fintech - % of non-minority using fintech"] 
    
    # Reverse Beta4
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 1) & (df_pred["ReverseBeta4"] >= 0)]
    df_obs.loc["ReverseBeta4", "% of minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 0) & (df_pred["ReverseBeta4"] >= 0)]
    df_obs.loc["ReverseBeta4", "% of non-minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_obs.loc["ReverseBeta4", "% of minority using fintech - % of non-minority using fintech"] = df_obs.loc["ReverseBeta4", "% of minority using fintech"] - df_obs.loc["ReverseBeta4", "% of non-minority using fintech"] 
    df_obs.loc["ReverseBeta4", "Obs. – Counter."] = df_obs.loc["ReverseBeta4", "% of minority using fintech - % of non-minority using fintech"] - df_obs.loc["obs", "% of minority using fintech - % of non-minority using fintech"] 
    
    # Shut off Beta1
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 1) & (df_pred["ShutoffBeta1"] >= 0)]
    df_obs.loc["ShutoffBeta1", "% of minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 0) & (df_pred["ShutoffBeta1"] >= 0)]
    df_obs.loc["ShutoffBeta1", "% of non-minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_obs.loc["ShutoffBeta1", "% of minority using fintech - % of non-minority using fintech"] = df_obs.loc["ShutoffBeta1", "% of minority using fintech"] - df_obs.loc["ShutoffBeta1", "% of non-minority using fintech"] 
    df_obs.loc["ShutoffBeta1", "Obs. – Counter."] = df_obs.loc["ShutoffBeta1", "% of minority using fintech - % of non-minority using fintech"] - df_obs.loc["obs", "% of minority using fintech - % of non-minority using fintech"] 
    
    # Shut off Beta2
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 1) & (df_pred["ShutoffBeta2"] >= 0)]
    df_obs.loc["ShutoffBeta2", "% of minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 0) & (df_pred["ShutoffBeta2"] >= 0)]
    df_obs.loc["ShutoffBeta2", "% of non-minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_obs.loc["ShutoffBeta2", "% of minority using fintech - % of non-minority using fintech"] = df_obs.loc["ShutoffBeta2", "% of minority using fintech"] - df_obs.loc["ShutoffBeta2", "% of non-minority using fintech"] 
    df_obs.loc["ShutoffBeta2", "Obs. – Counter."] = df_obs.loc["ShutoffBeta2", "% of minority using fintech - % of non-minority using fintech"] - df_obs.loc["obs", "% of minority using fintech - % of non-minority using fintech"] 
    
    # Shut off Beta3
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 1) & (df_pred["ShutoffBeta3"] >= 0)]
    df_obs.loc["ShutoffBeta3", "% of minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 0) & (df_pred["ShutoffBeta3"] >= 0)]
    df_obs.loc["ShutoffBeta3", "% of non-minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_obs.loc["ShutoffBeta3", "% of minority using fintech - % of non-minority using fintech"] = df_obs.loc["ShutoffBeta3", "% of minority using fintech"] - df_obs.loc["ShutoffBeta3", "% of non-minority using fintech"] 
    df_obs.loc["ShutoffBeta3", "Obs. – Counter."] = df_obs.loc["ShutoffBeta3", "% of minority using fintech - % of non-minority using fintech"] - df_obs.loc["obs", "% of minority using fintech - % of non-minority using fintech"] 
    

In [374]:
df_obs

,% of minority using fintech,% of non-minority using fintech,% of minority using fintech - % of non-minority using fintech,Obs. – Counter.
obs,0.038966,0.054701,-0.015735,NaN
ShutoffBeta4,0.871822,0.869444,0.002378,0.018113
